In [15]:
"""
- Download data
- Zoekt naar concidenties tussen STATIONS met een variabele 'coincidence time window'. 
- Berekend de richting van afkomst
- Plot de afkomstrichting

TO DO:
- 'Coincidence time window' automatisch berekenen voor de gebruikte stations n.a.v. de onderlinge afstand.
"""

import tables
import datetime
import sapphire as s
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import sys
from sapphire.utils import pbar
from sapphire.transformations.celestial import zenithazimuth_to_equatorial

global data, show_events, DATAFILE, events

def bewerk_data():
    global DATAFILE, events
    t3 = time.time()

    try:
        data = tables.open_file(DATAFILE, 'r+')
    except:
        print('Could not open data file, exit()')
        sys.exit()
        pass

    t4 = time.time()
    print('Opening data took: %.5f' % (t4-t3))
    print('Aantal coincidenties: %s' % len(data.root.coincidences.coincidences))
    print("Aantal reconstructions: %d " % (len(data.root.coincidences.reconstructions)))

    recs = data.root.coincidences.reconstructions.read()

    data.close()
    theta = recs['zenith']
    recs = recs.compress(~np.isnan(theta))
    
    if len(recs) == 0:
        print('Recs == 0, exit()')
        sys.exit()

    t5 = time.time()
    print('Removing NaNs from recs[theta] took: %.5f' % (t5-t4))

    print("Aantal reconstructions : %.2f " % (len(recs)))

    lla = s.HiSPARCStations(STATIONS).get_lla_coordinates()
    lat, lon, alt = lla

    t6 = time.time()
    print('get_lla_coordinates() took: %.5f' % (t6-t5))

    events = []
    for rec in pbar(recs):
        # omzetten naar 1 functie die matrix rekening doet? 1.5mil punten duurd 3 minuten
        timestamp = rec['ext_timestamp'] / 1.e9
        theta = rec['zenith']
        phi = rec['azimuth']
        r, d = zenithazimuth_to_equatorial(lat, lon, timestamp, theta, phi)  # Zelf maken zodat het sneller gaat?
        events.append((r-np.pi, d))
    events = np.array(events)

    t7 = time.time()
    print('Creating events = np.array(events) took: %.5f' % (t7-t6))

    ra = np.degrees(events[:, 0])

    dec = np.degrees(events[:, 1])

    t8 = time.time()
    print('RA & DEC naar degrees omzetten took: %.5f' % (t8-t7))


    t9 = time.time()
    print('Total runtime: %.2f' % (t9-t8))

    np.savetxt(file_name+'.csv', events, delimiter="\t")

    print('-----READY-----')

def plot_events_on_mollweide(events, filename=None):
    # Plot events (een lijst van RA, DEC tuples) op een kaart in Mollweide projectie"""

    # Let op: De RA-as is gespiegeld. Alle RA coordinates worden gespiegeld (negatief)
    # geplot.
    # RA, DEC tuples van het steelpan asterisme in het sterrenbeeld Grote Beer
    """
    RA en DEC zijn vindbaar met Aladin.
    Coordinaten daarvandaan pakken en dan een functie maken voor het omzetten naar graden?
    """
    global show_events
    steelpan = np.array([[13.792222, 49.3167], [13.398889, 54.9333], [12.900556, 55.95],
                         [12.257222, 57.0333], [11.896944, 53.7000], [11.030833, 56.3833],
                         [11.062222, 61.7500], [12.257222, 57.0333]])

    # Melkweg contouren als lijst van RA, DEC paren.
    # `milky_way.npy` heeft *geen* verbinding tussen RA 23h59 en 0h00 en `milky_way_polar.npy` wel.
    try:
        mw_contour = np.load('data\\numpy\\milky_way.npy')
        print('Loaded .npy files')
    except:
        mw_contour = []
        print('Failed to load .npy files')

    events = np.array(events)

    fig = plt.figure(figsize=(15, 15))
    #fig = plt.figure()
    ax = fig.add_subplot(111, projection="mollweide")
    
        # plot milky way contours
    for ra_mw, dec_mw in mw_contour:
        ax.plot(-ra_mw, dec_mw, color='grey')
    
    # let op: De RA as is gespiegeld:
    ax.set_xticklabels(['22', '20', '18', '16', '14', '12', '10', '8,0', '6,0', '4,0', '2,0'], fontsize='large')
    ax.set_yticklabels(['-75', '-60', '-45', '-30', '-15', '0', '15', '30', '45', '60', '75'], fontsize='large')
    ax.grid(True)
    ax.tick_params(axis='x', colors='white')
    ax.xaxis.label.set_color('white')
    ax.xaxis.set_label_coords(.5, .49)

    """
    Plot bron:
    https://python-graph-gallery.com/85-density-plot-with-matplotlib/
    """
    from scipy.stats import kde
    x = -events[:, 0]
    y = events[:, 1]

    
    # Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents
    t10 = time.time()
    
    nbins = 200
    k = kde.gaussian_kde([x, y])
    
    t11 = time.time()
    print('kde.gaussian_kde: %.2f s' % (t11-t10))
    
    xi, yi = np.mgrid[x.min():x.max():nbins * 1j, y.min():y.max():nbins * 1j]
    
    t12 = time.time()
    print('np.mgrid: %.2f s' % (t12-t11))
    
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))
    
    t13 = time.time()
    print('np.vstack: %.2f s' % (t13-t12))
    """ 
    Colormaps: https://matplotlib.org/examples/color/colormaps_reference.html
    jet

    """
    plt.pcolormesh(xi, yi, zi.reshape(xi.shape), cmap=plt.cm.jet, alpha=1)
    
    t14 = time.time()
    print(' plt.pcolormesh: %.2f s' % (t14-t13))
    
    plt.colorbar(shrink=0.5, pad=0.01)
    if show_events:
        ax.scatter(-events[:, 0], events[:, 1], marker='x', alpha=.99, color='grey', label='events')

    # plot steelpan in UMa
    ra_uma = np.radians(steelpan[:, 0] / 24 * 360 - 180.)
    dec_uma = np.radians(steelpan[:, 1])
    ax.plot(-ra_uma, dec_uma, color='white')
    ax.scatter(-ra_uma, dec_uma, color='white', s=10)

    # plot Polaris
    #ax.scatter(0., np.radians(90.), color='white', marker='*')
    #ax.text(np.radians(2.), np.radians(78.), 'Polaris', color='white', fontsize='10')

    # plot Galactic Center (RA 17h45, DEC -29)
    #ax.scatter(-np.radians(17.75 / 24 * 360 - 180.), np.radians(-29), color='white', marker='*')
    #ax.text(-np.radians(17.75 / 24 * 360 - 180. +2.), np.radians(-29 - 6.), 'Galactic Center', color='white', fontsize='10')

    plt.grid(alpha=.3)
    plt.xlabel('Rechte klimming [h]', fontsize='large')
    plt.ylabel('Declinatie [°]', fontsize='large')
    plt.tight_layout()
    #plt.legend()

    if filename:
        plt.savefig(filename, dpi=200, bbox_inches='tight')
    plt.show()

def refresh_data():
    global data
    try: 
        data.flush()
        data.close()
    except:
        pass
    data = tables.open_file(DATAFILE, 'a')

print('-----READY-----')

-----READY-----


In [16]:
t0 = time.time()

STATIONS = [2003, 2004, 2005, 2002] # Nijmegen
#STATIONS = [501, 502, 503]
START = datetime.datetime(2000, 1, 1)
END = datetime.datetime(2018, 6, 2)
N = 4
file_name = 'tele2'
#DATAFILE = 'data\\download\\'+file_name+'.h5'
DATAFILE = 'F:\\HiSPARC\\download\\'+file_name+'.h5'

overwrite = False

try:
    data.close()
    print('Data file closed')
except Exception as e:
    print('Could not close data file: %s' % e)
    pass

if __name__ == '__main__':
    if overwrite:
        try:
            os.remove(DATAFILE)
            print('Deleted data file')
        except Exception as e: 
            print('Could not delete data file: %s' % e) # Print de error die plaatsvindt
    data = tables.open_file(DATAFILE, 'a')
    station_groups = ['/station_%d' % u for u in STATIONS] # Deze 4 regels kunnen netter
    if overwrite:
        for station, group in zip(STATIONS, station_groups):
            s.download_data(data, group, station, START, END)

#print('Number of events for station %s: %s' %(2003, len(data.root.station_2003.events)))
t1 = time.time()
print('Downloading took: %.2f s' % (t1-t0))
print('----- READY -----')

Data file closed
----- READY -----


In [20]:
t2 = time.time()

"""
Herlaadt data zodat '/coincidences' in erin komt te staan, indien deze cell eerder is afgespeelt.
Vervolgens word deze group uit '/' (root) verwijderd, ze zijn tenslotte ''oud''.
Tot slot worden de coincidenties met de huidige instellingen gedownload.
"""
if '/coincidences' in data:
    try:
        refresh_data()
        data.remove_node('/',name='coincidences', recursive=True)
    except:
        pass

with s.CoincidencesESD(DATAFILE, '/coincidences', station_groups, overwrite=True) as coin:
    coin.search_coincidences(window=50000) # window – the coincidence time window in nanoseconds.  Origineel 5000
    coin.store_coincidences(STATIONS)

print('Number of coincidences: %s' % len(data.root.coincidences.coincidences))
t3 = time.time()
print('Searching and storing coincidences took: %.2f s' % (t3-t2))
print('----- READY -----')

100%|############################################################|Time: 0:10:09
100%|############################################################|Time: 0:06:14
100%|############################################################|Time: 0:00:18


Number of coincidences: 26650
Searching and storing coincidences took: 1914.73 s
----- READY -----


In [21]:
t0 = time.time()

try:
    refresh_data()
    data.remove_node('/coincidences',name='reconstructions', recursive=True)
except:
    pass

rec = s.analysis.reconstructions.ReconstructESDCoincidences(data, verbose=True, overwrite=True)
rec.reconstruct_and_store()

t1 = time.time()
print('Number of reconstructions: %s' % len(data.root.coincidences.reconstructions))
print('Creating reconstructions took: %.2f s' % (t1-t0))
print('----- READY -----')

c:\program files\python36\lib\site-packages\sapphire\clusters.py:873: UserWarning: Could not get detector layout for stations [2003, 2004, 2005, 2002], defaults will be used!
  'defaults will be used!' % str(missing_detectors))


Constructed cluster HiSPARCStations([2003, 2004, 2005, 2002]) from public database.


N/A%|                                                           |ETA:  --:--:--

Using timing offsets from public database.


c:\program files\python36\lib\site-packages\sapphire\analysis\event_utils.py:101: RuntimeWarning: All-NaN axis encountered
  station))
100%|############################################################|Time: 0:06:38
N/A%|                                                           |ETA:  --:--:--c:\program files\python36\lib\site-packages\sapphire\analysis\event_utils.py:33: RuntimeWarning: Mean of empty slice
  station=station))
100%|############################################################|Time: 0:01:23


Number of reconstructions: 26650
Creating reconstructions took: 817.86 s
----- READY -----


In [22]:
bewerk_data()

Opening data took: 0.02755
Aantal coincidenties: 26650
Aantal reconstructions: 26650 
Recs == 0, exit()


SystemExit: 

c:\program files\python36\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
show_events = False
events = np.loadtxt(file_name+".csv")
events = np.append(events, [[-1.57, -1.57]], axis=0) # een punt toevoegen zodat de gehele plot ingekleurd is.
t15 = time.time()

plot_events_on_mollweide(events, filename='figuren\\'+file_name+'.png')

t16 = time.time()
print('Plotting took: %.2f s' % (t16-t15))

In [ ]:
def calc_time_window():
    """
    Zie klad eigen: 5000 ns is voor een shower van ~60 graden (totaal dus 2*60 graden)
    Neem x_prime lorentz transformatie, haal x eruit en vul in:
    t' = levensduur muon (komt voort uit schatting met t' berekenen, t=5000ns x=200)
    t = 5000 ns
    """
    afstanden = [500, 10, 1000]
    x = max(afstanden)
    
    c = 299792458
    v = 0.98*c
    t_prime = 2.1969811*10**-6 # Muon levensduur
    gamma = 1/np.sqrt(1-(v**2/c**2))
    t=(t_prime+v*x/c)*1/gamma
    return t
calc_time_window()